<p style="font-family: Arial; font-size:3em;color:black;"> Session 4 - Lab Exercise 7</p>

We will use the titanic dataset from Kaggle (https://www.kaggle.com/). This is a well-known dataset and we will use it for classification- if the passenger survived or passed away.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
training_DF = pd.read_csv('titanic_dataset_GBC.csv')

In [3]:
training_DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
training_DF['Age'].fillna(training_DF['Age'].mean(), inplace=True)

In [5]:
training_DF.drop('Cabin',axis=1,inplace=True)

In [6]:
training_DF.dropna(inplace=True)

In [7]:
sex = pd.get_dummies(training_DF['Sex'])
embark = pd.get_dummies(training_DF['Embarked'])

In [8]:
training_DF.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [9]:
training_DF = pd.concat([training_DF,sex,embark],axis=1)

In [10]:
training_DF.drop(['female','C'],axis=1,inplace=True)

In [11]:
training_DF.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,1,0,3,22.0,1,0,7.2500,True,False,True
1,2,1,1,38.0,1,0,71.2833,False,False,False
2,3,1,3,26.0,0,0,7.9250,False,False,True
3,4,1,1,35.0,1,0,53.1000,False,False,True
4,5,0,3,35.0,0,0,8.0500,True,False,True


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(training_DF.drop('Survived',axis=1), 
                                                    training_DF['Survived'], test_size=0.30, 
                                                    random_state=101)

In [14]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [15]:
# let's explore SGDClassifier parameters

'''
SGDClassifier(
    loss='hinge',
    penalty='l2',
    alpha=0.0001,
    l1_ratio=0.15,
    fit_intercept=True,
    max_iter=1000,
    tol=0.001,
    shuffle=True,
    verbose=0,
    epsilon=0.1,
    n_jobs=None,
    random_state=None,
    learning_rate='optimal',
    eta0=0.0,
    power_t=0.5,
    early_stopping=False,
    validation_fraction=0.1,
    n_iter_no_change=5,
    class_weight=None,
    warm_start=False,
    average=False,
)
'''

# Let's form SGD models with variation in paameters loss and alpha
    # loss: 'hinge', 'log', and 'modified_huber'
    # alpha: 0.0001 and 0.001
    # explain and dicuss your findings

"\nSGDClassifier(\n    loss='hinge',\n    penalty='l2',\n    alpha=0.0001,\n    l1_ratio=0.15,\n    fit_intercept=True,\n    max_iter=1000,\n    tol=0.001,\n    shuffle=True,\n    verbose=0,\n    epsilon=0.1,\n    n_jobs=None,\n    random_state=None,\n    learning_rate='optimal',\n    eta0=0.0,\n    power_t=0.5,\n    early_stopping=False,\n    validation_fraction=0.1,\n    n_iter_no_change=5,\n    class_weight=None,\n    warm_start=False,\n    average=False,\n)\n"

In [16]:
from sklearn.model_selection import GridSearchCV 

In [17]:
sgdc = SGDClassifier()

In [18]:
param_grid = {'loss':['hinge','log_loss','modified_huber'], 'alpha':[0.0001,0.001]}
#It raised an error related to the 'log' value in the 'loss' parameter .
#With a newer version, it probably has changed to the 'log_loss' option.

In [19]:
gridCV = GridSearchCV(sgdc, param_grid)

In [20]:
gridCV.fit(X_train,y_train)

GridSearchCV(estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001],
                         'loss': ['hinge', 'log_loss', 'modified_huber']})

In [21]:
print(f'Best parameters for our model: {gridCV.best_params_}')
print(f'The accuracy score:{gridCV.best_score_}')

Best parameters for our model: {'alpha': 0.001, 'loss': 'modified_huber'}
The accuracy score:0.7927096774193549


In [22]:
from sklearn.metrics import confusion_matrix , classification_report

#### Since we scaled the data for training, we need to scale the test data too

In [23]:
X_test = scaler.fit_transform(X_test)

In [24]:
y_predictions = gridCV.predict(X_test)

In [25]:
confusion_matrix(y_test, y_predictions)

array([[140,  23],
       [ 29,  75]])

In [26]:
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       163
           1       0.77      0.72      0.74       104

    accuracy                           0.81       267
   macro avg       0.80      0.79      0.79       267
weighted avg       0.80      0.81      0.80       267



In [30]:
#For class 0: Precision is 0.83, indicating that 83% of the predictions for class 0 were correct. 
#Recall is 0.86, suggesting that the model is good at capturing class 0 instances.

#For class 1: Precision is 0.77, indicating that 77% of the predictions for class 1 were correct. 
#However, recall is 0.72, which means the model is less effective at capturing class 1 instances.

In [31]:
#This difference in precision and recall scores between 0 and 1 labels
# can be a result of unbalanced dataset where number of samples with label 0
# is higher than number of samples with label 1
training_DF['Survived'].value_counts()

Survived
0    549
1    340
Name: count, dtype: int64

In [28]:
pd.DataFrame(gridCV.cv_results_).sort_values('rank_test_score').iloc[:,4:]

,param_alpha,param_loss,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.001,modified_huber,"{'alpha': 0.001, 'loss': 'modified_huber'}",0.696,0.824,0.782258,0.822581,0.838710,0.792710,0.051864,1
3,0.001,hinge,"{'alpha': 0.001, 'loss': 'hinge'}",0.728,0.832,0.782258,0.766129,0.846774,0.791032,0.043487,2
4,0.001,log_loss,"{'alpha': 0.001, 'loss': 'log_loss'}",0.720,0.816,0.790323,0.774194,0.806452,0.781394,0.033842,3
1,0.0001,log_loss,"{'alpha': 0.0001, 'loss': 'log_loss'}",0.736,0.832,0.782258,0.709677,0.741935,0.760374,0.042693,4
2,0.0001,modified_huber,"{'alpha': 0.0001, 'loss': 'modified_huber'}",0.720,0.808,0.774194,0.806452,0.669355,0.755600,0.053634,5
0,0.0001,hinge,"{'alpha': 0.0001, 'loss': 'hinge'}",0.728,0.824,0.588710,0.709677,0.814516,0.732981,0.085223,6


In [29]:
# It can be easily seen that 0.001 value of alpha gave the top three accuracy scores.
# And the alpha = 0.0001 with modified_huber loss function gave the worst result.